<center><a href="https://www.nvidia.cn/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

# <font color="#76b900">**3:** LLM 编码任务</font>

在上个 notebook 中，您更深入的了解了 HuggingFace &#x1F917; pipeline，知道 LLM 是如何进行理自然语言推理的。在这个 notebook 中，我们将把 BERT 编码器应用到去掩码任务以外的任务上。

#### **学习目标：**

* 了解如何使用任务特定的 pipeline 来处理 token 级、文章级和范围截取（range-subsetting）的任务。
* 用相同的抽象实现零样本分类，无需重新训练即可分类任意类别。

---

## 3.1. 用于预测 token 的任务头

之前，我们看到了去掩码 pipeline，稍微探索了一下它的运作方式。现在我们回顾一下，看看模型中发生了什么：

In [ ]:
# from transformers import pipeline
# unmasker = pipeline('fill-mask', model='bert-base-uncased')

from transformers import BertTokenizer, BertModel, FillMaskPipeline, AutoModelForMaskedLM

unmasker = FillMaskPipeline(
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased'),
    model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
)
unmasker("Hello, Mr. Bert! How is it [MASK]?")

们之前讨论过 LLM 进行推理的方式：

* 学习能够捕捉序列项语义和位置信息的编码形式。
* 提供了跨 token 推理的接口（也就是注意力），同时还能大体上保证 token 的语义边界（用残差连接）。

这是 **transformer 编码器**的默认设置，用于将输入序列转换成隐式序列：

* token 序列输入。
* 序列在网络中传播。
* 输出一系列向量。

知道了这些，去掩码背后的机制就应该非常直观了：如果 transformer 层的输出具有丰富的语义和上下文信息，那我们只需通将每个嵌入的序列条目送到密集网络，就可以进行 **token 级预测**了！

**用这种粒度预测 token 类别的任务就被称为 [token 分类](https://huggingface.co/tasks/token-classification)！**我们看到的[掩码填充](https://huggingface.co/tasks/fill-mask)，就是当输出从类别空间变为 token 空间时，token 分类的另一种形式。

![Task as seen on https://huggingface.co/tasks/token-classification](imgs/task-token-classification.png)

知道了这点后，我们再来看一下 `(cls)`，也就是掩码模型的**分类头**组件：

```python
unmasker.model.cls
```

```python
BertOnlyMLMHead(
  (predictions): BertLMPredictionHead(
    (transform): BertPredictionHeadTransform(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (transform_act_fn): GELUActivation()
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    )
    (decoder): Linear(in_features=768, out_features=30522, bias=True)
  )
)
```

与之前一样，您可以随时查看[源代码](https://github.com/huggingface/transformers/blob/7a6efe1e9f756f585f2ffe5ada22cf6b15edd23b/src/transformers/models/bert/modeling_bert.py#L686)，看看它到底是如何实现的，逻辑很简单：

* BERT 固定输出 768 维的向量，每个向量依次通过带有 [GELU 激活](https://pytorch.org/docs/stable/generated/torch.nn.GELU.html)和[层归一化](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html) 的密集层。
	+ **GELU（高斯误差线性单元）：**与 ReLU 类似，但更平滑一些。
	+ **LayerNorm（层归一化）：**之前已讨论过。帮助归一化输出，使优化更平滑。
* 最后再通过一个密集层将隐式表达转为可能的 token 集上的概率分布。

换言之，就是为分类器提供一些非线性推理能力，以预测最终的输出 token。还不错，对吧？理论上，这应该足以为每个输入 token 预测新的输出 token！为了验证维度是否符合预期，我们可以在 pipeline 的前向传播过程中打印出中间过程：

In [ ]:
from transformers import BertTokenizer, BertModel, FillMaskPipeline, AutoModelForMaskedLM

class MyFillMaskModel(FillMaskPipeline):
    def __init__(self):
        super().__init__(
            tokenizer = BertTokenizer.from_pretrained('bert-base-uncased'),
            model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased")
        )

    def __call__(self, string):
        # input_tensors = self.preprocess(string)
        # output_tensors = self.forward(input_tensors)
        # output = self.postprocess({**input_tensors, **output_tensors})

        input_tensors = unmasker.preprocess("I really wish [MASK] were an instrument!")

        inputs = {'input_ids' : input_tensors['input_ids']}
        x = unmasker.model.bert.embeddings.forward(**inputs)
        print("Shape from embedding into encoder:", x.shape)

        inputs = {'attention_mask' : input_tensors['attention_mask']}
        x = unmasker.model.bert.encoder.forward(x, **inputs)
        print("Shape from encoder into cls:", x['last_hidden_state'].shape)

        x = unmasker.model.cls.forward(x['last_hidden_state'])
        print("Shape from cls into postprocess:", x.shape)

        output = self.postprocess({**input_tensors, 'logits' : x})

        return output


unmasker = MyFillMaskModel()
unmasker("I really wish [MASK] were an instrument!")[0]

可以看到为了使 pipeline 能正常工作，`postprocess` 阶段执行了一些额外工作。您可以随时查看[源代码](https://github.com/huggingface/transformers/blob/95b374952dc27d8511541d6f5a4e22c9ec11fb24/src/transformers/pipelines/fill_mask.py#L105)了解具体是怎么运行的。您将不会感到惊讶，postprocess 是通过找到字符串中的第一个 `[MASK]`，并查看预测的概率向量中哪些条目值最大来起作用的（就是 **argmax-ing** 操作，用索引作为参数）。

## 3.2. 范围输出的 Token 级预测

上述任务是一个自然的 token 输入输出示例，适用于大多数类 BERT 模型的主要训练目标：

* [**掩码语言模型 (MLM)**](https://huggingface.co/docs/transformers/main/tasks/masked_language_modeling)
	+ **训练目标：**恢复原始 token。
	+ **增强：**在训练数据中，将部分 token 替换为 [MASK]，再将更多的其它 token 替换为随机 token。
	+ **目标：**获得 token 级任务的双向推理能力。

然而，BERT 模型绝不仅限于这类任务！（尽管您的分类器会是这样的）预训练 LLM 的目的是将它们用作语言理解的主干，因此我们应该能向基础模型添加不同的头以完成其它类型的任务。

一个稍有不同的 token 级任务就是范围预测任务！在此任务中，分类器训练后可以通过预测输入序列的一个开始位置和一个结束位置所截取的子集作为响应。“你得到的回答是输入的一部分”，这样的性质实际上非常适合用来限制较小模型的推理空间，因此它成了[问答](https://huggingface.co/tasks/question-answering)任务中常用的方式。

![Task as seen on https://huggingface.co/tasks/question-answering](imgs/task-qa.png)

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

## Example from https://huggingface.co/deepset/roberta-base-squad2
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
nlp(QA_input)

RoBERTa 是与 BERT 类似的另一种仅使用编码器的模型，所以它的输入输出也应该是相同的。我们只需要了解，它们的主要区别在于 RoBERTa 的分类头（基于[SQuAD2.0 问答数据集](https://huggingface.co/datasets/squad_v2)进行了微调）针对每个 token 仅预测两个值：作为开头的概率和作为结尾的概率。

可以通过查看新的分类头来验证这一点，您可能会发现这比去掩码的分类头简单很多：

```python
nlp.model.qa_outputs
```
> ```python
(qa_outputs): Linear(in_features=768, out_features=2, bias=True)
```

在 $768 \to 2$ 的维度转换后，后处理只需找出开始和结束序列的理想范围（即最大化两个预测值的和），我们就得到了子字符串预测模型！

这种模式稍加修改，就可以应用于文本摘要或任何其它子字符串任务，并且它的优劣势很明确：预测被严格的限制为输入的子集！当您的应用注重稳定性并需要规避风险的时候（比如面向公众的应用），这一点尤为重要。但它也可以被视为一个缺点，因为这样的限制使模型无法以对话形式进行输出。

## 3.3. 序列级分类头

回想一下，除了 BERT 中的掩码语言模型任务之外，模型还在 NSP（Next-Sentence Prediction）任务上进行了训练：

* **下一句预测（NSP）**
	+ **训练目标：**预测句子 A 是否在句子 B 之后
	+ **增强：**将句子对组合在一起，50% 的概率 A 和 B 按数据集中的顺序出现。
	+ **目标：**实现长跨度推理，为第一个 token 赋予一般分类的能力。

**MLM** 的目标是许多类 BERT 大语言模型的核心，而 **NSP** 任务自从纳入 BERT 以来就一直备受争议。将跨度更长的逻辑整合到模型输出的特定部分确实有一定道理，这将提高模型的语言推理能力。但后续的架构尝试去掉这个目标（请参阅[**RoBERTa**](https://huggingface.co/docs/transformers/model_doc/roberta)）或替换为其它实现（请参阅[**Albert**](https://huggingface.co/albert-base-v2)）。无论使用哪种泛化技术来训练 transformer 编码器模型，分类头的工作流程通常都非常一致：

> **取输出中的一个特定项（比如说第一项或者是 `CLS` 项）经过一系列密集层，来构造出想要的输出形式。**

**这被称为[文本分类](https://huggingface.co/tasks/text-classification)！**（或更一般的称为“序列分类”）

<div><img src="imgs/task-text-classification.png" 
     alt="Task as seen on https://huggingface.co/tasks/text-classification"
     width="800"/></div>

这个范式对段落级推理非常有效，让我们用一个很流行的情感分类模型来说明这一点：

In [ ]:
from transformers import AutoModelForSequenceClassification

emo_model = pipeline('sentiment-analysis', 'SamLowe/roberta-base-go_emotions')

print(emo_model("I love my old pillow?"))
print(emo_model("Why is it that every plant I touch dies within a few days?"))
print(emo_model("I'm so conflicted about these new instructions..."))

我们可以看到，尽管 RoBERTa 不依赖于下一句预测任务，它的效果依然非常好。在使用互联网级别的大量数据训练时，它只需要用一些分类监督信号，就可以让整个网络捕捉段落级的关系。

在研究架构时，您可能会发现从 token 级分类头到序列级分类头的切换并不明显：

```python
emo_model.model.classifier
```
> ```python
RobertaClassificationHead(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (out_proj): Linear(in_features=768, out_features=28, bias=True)
)
```

这看起来与我们之前看到的 token 级预测任务几乎相同。要看到对句子进行截取的实际运行情况，您确实需要查看源代码！幸运的是，分类器的打印输出确实表明此逻辑包含在 `RobertaClassificationHead` 之中。我们只需要从[官方源代码](https://github.com/huggingface/transformers/blob/f26099e7b5cf579f99a42bab6ddd371bf2c8d548/src/transformers/models/roberta/modeling_roberta.py#L1510)找找看发生了什么：

```python
class RobertaForQuestionAnswering(RobertaPreTrainedModel):
    # ...
    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        token_type_ids: Optional[torch.LongTensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        start_positions: Optional[torch.LongTensor] = None,
        end_positions: Optional[torch.LongTensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], QuestionAnsweringModelOutput]:
        ## As you can see, the forward call starts off by passing
        ## the inputs through the base model.
        outputs = self.roberta(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        ## Then, it just has to take the first sequence entry
        ## from the model push it through the dense layers
        ## for a single set of classifications.
        sequence_output = outputs[0]   ### <- Interesting
        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1).contiguous()
        end_logits = end_logits.squeeze(-1).contiguous()
        ## ...
```

能够深入到这种开源项目的底层对于理解最前沿的软件原理是至关重要的，对于您认为值得探究一下的困惑点，别忘了去翻翻代码库！

## 3.4. 零样本分类

到目前为止，我们一直在讨论怎么调整编码器，使得**只调用一次函数**就能得到不同的输出。实际上，允许模型被多次调用，就可以摆脱 `n->1` 和 `n->n` 这两种模式。

在后面的 notebook 中，我们将了解 transformer 解码器是如何以自回归的方式预测整个序列的，一次输入一个 token。这需要特殊的训练方式，但现在我们可以先试试：

> **预测类别的概率，一次一个类别，直到对您想了解的所有类别都进行完预测！**

**这就叫[零样本分类](https://huggingface.co/tasks/zero-shot-classification)！**

<div><img src="imgs/task-zero-shot.png" 
     alt="Zero-shot classification task as seen on https://huggingface.co/tasks/zero-shot-classification"
     width="800"/></div>

如果您之前没接触过这个领域，可能不太熟悉零样本这个术语，下面我们快速定义一些关键的概念：

> * **零样本推理**：让模型预测从未经过专门训练的事物。
> * **少样本推理**：让模型预测训练中或上下文中出现过的内容，但这个方面的训练数据量非常有限。

您会注意到，从现在起，本课程中的大多数事情都将是零样本的，我们要求模型做的事情未必已经过训练。它能做到这些，是因为它对语言和词的含义有了一定的“理解”。

#### **任务 1：**零样本 Pipeline

浏览任务说明并导入模型（[`facebook/bart-large-mnli`](https://huggingface.co/facebook/bart-large-mnli)）！用几个例子试试，可以先从上面的示例开始，然后试试您真正感兴趣的。

In [ ]:
## Your Code Here:

完成之后，猜猜这些值是如何产生的，看看能不能找到能验证您猜想的资料！（可能在源代码里，也可以仿照 pipeline 自己写一个，或者也可能在其它的什么地方）

**提示：**

* 考虑一下这个模型是用什么任务训练的：[MultiNLI](https://huggingface.co/datasets/multi_nli)
* 可以参考模型卡片，手写一个 pytorch 实现可能会有帮助。
* 如果您在使用 pipeline 时被卡住了，请随时查看 [`99_licenses.ipynb` notebook](extras_and_licenses/99_licenses.ipynb)，里面展示了几种推荐模型默认 API 用法。

## 3.5. 总结

现在，我们已经了解了如何使用相对一致的 transformer 编码器架构来执行一些关键的任务：

* **Token 级预测**
	+ 为序列中的每个条目做预测。
	+ 非常适合 1 对 1 的 token 转换或选择任务，包括用于截取输入子集的**范围预测**。

* **序列级预测**
	+ 通过从特定序列条目中提取值和语义来理解段落级的数据。
	+ 非常适合语义分析和段落分类。

* **多问题预测**
	+ 多次调用编码器架构来生成多个序列级预测。
	+ 可以不停的生成结果，一次生成一个。

**在下一个 notebook 中，我们将了解一些其它架构，这些架构扩展了多次预测的逻辑，能生成一个完整的有序序列（也就是，自然语言）！**

In [ ]:
## Please Run When You're Done!
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)